In [1]:
import sys
import time
import csv
import math
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
import json
from operator import add
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from node2vec import Node2Vec as n2v
import networkx as nx
from hashlib import md5
import pickle
import os
import pandas as pd
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 100)
pd.set_option("display.precision", 2)


from utils import create_category_md5_mapping, integrate_mapping_user_bus_cat_data, dataframe_to_rdd_dict, analyze_top_business_categories, analyze_top_categories

import json
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def initialize_spark_context(APP_NAME="Train: XGBModel"):
    # Spark配置项列表
    SPARK_CONF = [
        ("spark.dynamicAllocation.enabled", "true"),  # 启用动态资源分配
        ("spark.dynamicAllocation.maxExecutors", "10"),  # 最大执行器数量
        ("spark.executor.memory", "3g"),  # 每个执行器的内存
        ("spark.executor.cores", "2"),  # 每个执行器的CPU核心数
        ("spark.executor.memoryOverhead", "3000"),  # 执行器内存开销
        ("spark.driver.memory", "4g"),  # 驱动程序的内存
        ("spark.driver.maxResultSize", "2g"),  # 驱动程序的最大结果大小
        ("spark.python.worker.memory", "2g"),  # Python工作进程的内存
        ("spark.sql.shuffle.partitions", "20"),  # Shuffle操作的分区数
        ("spark.sql.sources.partitionOverWriteMode", "dynamic"),  # 分区覆写模式
        ("spark.network.timeout", "600s"),  # 网络超时设置
        ("spark.executor.heartbeatInterval", "120s"),  # 执行器心跳间隔
    ]

    # 创建Spark配置
    spark_conf = pyspark.SparkConf()
    spark_conf.setAppName(APP_NAME)
    spark_conf.setAll(SPARK_CONF)

    # 创建SparkContext
    sc = pyspark.SparkContext(conf=spark_conf)
    sc.setLogLevel("ERROR")  # 设置日志级别

    return sc

sc = initialize_spark_context()

import better_features
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data, extract_review_data
from datetime import datetime #add
import time
import pandas as pd

def rdd_to_pandas(rdd):
    return pd.DataFrame(rdd.collect(), columns=rdd.first().keys())

def prepare_test_data(test_df, cluster):
    if cluster == -1:
        test_cluster_data = test_df
    else:
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    return X_test, y_test

start_time = time.time()

user_parsed_df = pd.read_csv('cache/user_df.csv') # parsed from users.json
business_parsed_df = pd.read_csv('cache/business_df.csv') # parsed from business.json
review_parsed_df = pd.read_csv('cache/review_df.csv') # parsed from business.json

train_df = pd.read_csv('cache/train_df.csv', index_col=None)
test_df = pd.read_csv('cache/test_df.csv', index_col=None)
val_df = pd.read_csv('cache/val_df.csv', index_col=None)

print("DONE!")

24/05/04 02:13:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


DONE!


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 55682)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lo

In [3]:
# from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data,extract_review_data
# from utils import integrate_mapping_user_bus_cat_data
# from KMeans_user_cluster import KMeans_process_user_clusters

# data_folder_path, test_data_file, output_file = '../data/', '../data/yelp_true.csv', 'prediction.csv'
# feature_processor = FeatureProcessor(sc, data_folder_path, user_parsed_df, business_parsed_df, review_parsed_df)
# user_clusters = KMeans_process_user_clusters(feature_processor.map_reviews_with_categories(), business_parsed_df)

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [4]:
# ## TRAIN DATA FEATURE PROCESSING HERE
# train_data_file = '../data/yelp_train.csv'
# train_df = pd.read_csv(train_data_file)
# train_df = feature_processor.process_all_features(sc, train_df, train_data_file)
# train_df = train_df.merge(user_clusters, on='user_id', how='left')

# print("DONE!")

# ## TEST DATA FEATURE PROCESSING HERE
# val_data_file = '../data/yelp_val.csv'
# val_df = pd.read_csv(val_data_file)
# val_df = feature_processor.process_all_features(sc, val_df, test_data_file)
# val_df = val_df.merge(user_clusters, on='user_id', how='left')

# print("DONE!")

# ## TEST DATA FEATURE PROCESSING HERE
# test_data_file = '../data/yelp_true.csv'
# test_df = pd.read_csv(test_data_file)
# test_df = feature_processor.process_all_features(sc, test_df, test_data_file)
# test_df = test_df.merge(user_clusters, on='user_id', how='left')

# print("DONE!")

DONE!


DONE!


DONE!


In [7]:
# train_df.to_csv('archived/train_df.csv', index=False)
# test_df.to_csv('archived/test_df.csv', index=False)
# val_df.to_csv('archived/val_df.csv', index=False)

# print("DONE!")

DONE!


In [2]:
# ALL use early_stopping_rounds=10

# 0, 2, 3, 4, 

# 0,2 no splitting val, 
# 3,4 using only catboost
# test_size = 0.05
large_xgb_params = {
    0: {'learning_rate': 0.04246778091879101, 'max_depth': 6, 'n_estimators': 491, 'subsample': 0.8689},
    2: {'learning_rate': 0.12350751460907078, 'max_depth': 3, 'n_estimators': 288, 'subsample': 0.8880},
    3: {'learning_rate': 0.08288064461501718, 'max_depth': 5, 'n_estimators': 305, 'subsample': 0.6839},
    4: {'learning_rate': 0.18299396047960448, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9081}
}

large_catboost_params = {
    0: {'depth': 12, 'l2_leaf_reg': 18.15, 'learning_rate': 0.06229, 'n_estimators': 1486},
    2: {'depth': 6, 'l2_leaf_reg': 28.02, 'learning_rate': 0.1090, 'n_estimators': 176},
    3: {'depth': 5, 'l2_leaf_reg': 0.2358, 'learning_rate': 0.1968, 'n_estimators': 62},
    4: {'depth': 4, 'l2_leaf_reg': 0.3376, 'learning_rate': 0.1914, 'n_estimators': 46}
}

# 5, 6, 7, 8
# test_size = 0.1
medium_catboost_params = {
    5: {'depth': 3, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
    7: {'depth': 6, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
    8: {'depth': 1, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126},
    6: {'depth': 2, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}
}

In [ ]:
# def prepare_data(df, cluster, drop_cols):
#     data = df[df['Cluster'] == cluster]
#     X = data.drop(columns=drop_cols)
#     y = data['stars']
#     return X, y

In [23]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.impute import SimpleImputer

def prepare_data(df, cluster, drop_cols):
    if cluster == -1:
        cluster_data = df
    else:
        cluster_data = df[df['Cluster'] == cluster]

    drop_cols = [col for col in drop_cols if col in cluster_data.columns]
    X = cluster_data.drop(columns=drop_cols, errors='ignore')
    y = cluster_data['stars']
    return X, y

def train_model(cluster, train_df, test_df):
    if cluster in [5]:
            drop_cols = ['stars', 'user_id', 'business_id', 'log_affinity_score']  # For cluster 5, do not drop 'score'
    elif cluster in [7, 8]:
        drop_cols = ['stars', 'user_id', 'business_id', 'score', 'log_affinity_score']  # Drop both 'score' and 'log_affinity_score'
    else:
        drop_cols = ['stars', 'user_id', 'business_id']  # Default columns to drop
    
    if cluster in [0, 2]:
        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)  

        cb_model = CatBoostRegressor(**large_catboost_params[cluster], verbose=False)
        xgb_model = XGBRegressor(**large_xgb_params[cluster], objective='reg:squarederror', verbosity=0)
        
        cb_model.fit(X_train, y_train)
        xgb_model.fit(X_train, y_train)
        
        cb_preds = cb_model.predict(X_test)
        xgb_preds = xgb_model.predict(X_test)
        
        # Applying best weight ratio and clipping
        final_preds = (0.30 * cb_preds + 0.70 * xgb_preds) if cluster == 0 else (0.65 * cb_preds + 0.35 * xgb_preds)
        final_preds = np.clip(final_preds, 1, 5)
    elif cluster in [3, 4, 5, 6, 7, 8]:
        test_size = 0.05 if cluster in [3, 4] else 0.10
        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=42)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)

        model = CatBoostRegressor(**(large_catboost_params[cluster] if cluster in [3, 4] else medium_catboost_params[cluster]), verbose=False)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)
        final_preds = model.predict(X_test).clip(1, 5)
    elif cluster == 1:
        if pd.isnull(train_df['score']).any() or pd.isnull(test_df['score']).any():
            drop_cols.append('score')
            
        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)

        # model = ElasticNet(**small_ES_params[cluster])
        model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=np.logspace(-6, 2, 100), cv=5, random_state=42)
        model.fit(X_train, y_train)
        
        model = ElasticNet(alpha=model.alpha_, l1_ratio=model.l1_ratio_)
        model.fit(X_train, y_train)
        final_preds = model.predict(X_test).clip(1, 5)

    
    rmse = np.sqrt(mean_squared_error(y_test, final_preds))
    return rmse


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.impute import SimpleImputer


def find_best_weight(cluster, X_train, y_train, X_test, y_test):
    # init
    cb_model = CatBoostRegressor(**large_catboost_params[cluster], verbose=False)
    xgb_model = XGBRegressor(**large_xgb_params[cluster], objective='reg:squarederror', verbosity=0)

    # train
    cb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)

    # predict
    cb_preds = cb_model.predict(X_test).clip(1, 5)
    xgb_preds = xgb_model.predict(X_test).clip(1, 5)

    best_rmse = float('inf')
    best_ratio = 0
    
    for ratio in np.linspace(0, 1, 21):  # test different porpotion
        final_preds = ratio * cb_preds + (1 - ratio) * xgb_preds
        rmse = np.sqrt(mean_squared_error(y_test, final_preds))
        if rmse < best_rmse:
            best_rmse = rmse
            best_ratio = ratio

    return best_ratio, best_rmse

In [97]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.impute import SimpleImputer


def find_best_weight(cluster, X_train, y_train, X_test, y_test):
    # init
    cb_model = CatBoostRegressor(**large_catboost_params[cluster], verbose=False)
    xgb_model = XGBRegressor(**large_xgb_params[cluster], objective='reg:squarederror', verbosity=0)

    # train
    cb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)

    # predict
    cb_preds = cb_model.predict(X_test).clip(1, 5)
    xgb_preds = xgb_model.predict(X_test).clip(1, 5)

    best_rmse = float('inf')
    best_ratio = 0
    
    for ratio in np.linspace(0, 1, 21):  # test different porpotion
        final_preds = ratio * cb_preds + (1 - ratio) * xgb_preds
        rmse = np.sqrt(mean_squared_error(y_test, final_preds))
        if rmse < best_rmse:
            best_rmse = rmse
            best_ratio = ratio

    return best_ratio, best_rmse


def train_model(cluster, train_df, test_df):
    def prepare_data(df, cluster, drop_cols):
        if cluster == -1:
            cluster_data = df
        else:
            cluster_data = df[df['Cluster'] == cluster]
    
        drop_cols = [col for col in drop_cols if col in cluster_data.columns]
        X = cluster_data.drop(columns=drop_cols, errors='ignore')
        y = cluster_data['stars']
        return X, y
        
    if cluster in [5]:
        drop_cols = ['stars', 'user_id', 'business_id', 'log_affinity_score']  # For cluster 5, do not drop 'score'
    elif cluster in [6, 7, 8]:
        drop_cols = ['stars', 'user_id', 'business_id', 'binary_affinity_score', 'log_affinity_score']  # Drop both 'score' and 'log_affinity_score'
    else:
        drop_cols = ['stars', 'user_id', 'business_id']  # Default columns to drop
    
    if cluster in [0, 2]:
        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)

        cb_model = CatBoostRegressor(**large_catboost_params[cluster], verbose=False)
        xgb_model = XGBRegressor(**large_xgb_params[cluster], objective='reg:squarederror', verbosity=0)
        
        cb_model.fit(X_train, y_train)
        xgb_model.fit(X_train, y_train)
        
        cb_preds = cb_model.predict(X_test)
        xgb_preds = xgb_model.predict(X_test)
        
        # Applying best weight ratio and clipping
        #  if cluster == 0 else (0.65 * cb_preds + 0.35 * xgb_preds)
        best_ratio, best_rmse = find_best_weight(cluster, X_train, y_train, X_test, y_test)
        final_preds = (best_ratio * cb_preds + (1 - best_ratio) * xgb_preds)
        final_preds = np.clip(final_preds, 1, 5)

    elif cluster in [1]:
        if pd.isnull(train_df['score']).any() or pd.isnull(test_df['score']).any():
            drop_cols.append('score')

        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)
        
        # model = ElasticNet(**small_ES_params[cluster])
        model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=np.logspace(-6, 2, 100), cv=5, random_state=42)
        model.fit(X_train, y_train)
        final_preds = model.predict(X_test).clip(1, 5)

    elif cluster in [3]:
        test_size = 0.25  
        X_train, y_train = prepare_data(train_df, cluster, drop_cols)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=42)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)
        model = CatBoostRegressor(**large_catboost_params[cluster], verbose=False)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30)
        final_preds = model.predict(X_test).clip(1, 5)
        
    elif cluster in [4, 5, 6, 7, 8]:
        test_size = 0.1
        X_train, y_train = prepare_data(test_df, cluster, drop_cols)
        X_test, y_test = prepare_data(test_df, cluster, drop_cols)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=42)
        
        model = CatBoostRegressor(**(large_catboost_params[cluster] if cluster in [3, 4] else medium_catboost_params[cluster]), verbose=False)
        model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=10)
        final_preds = model.predict(X_test).clip(1, 5)
        
    # merge a prediction df
    id_and_stars = test_df[test_df['Cluster'] == cluster][['user_id', 'business_id', 'stars']]
    predictions_df = pd.DataFrame(final_preds, index=id_and_stars.index, columns=['predicted_stars'])
    results_df = pd.concat([id_and_stars, predictions_df], axis=1)
    
    rmse = np.sqrt(mean_squared_error(y_test, final_preds))
    # rmse = np.sqrt(mean_squared_error(results_df['stars'], results_df['predicted_stars']))
    return results_df, rmse

In [100]:
clusters_rmse = {}
all_results = pd.DataFrame()

# for cluster in range(9):  # 假定有0到8个cluster
for cluster in [2,1,3,4,5,6,7,8]:
    result_df, rmse = train_model(cluster, train_df, val_df)
    clusters_rmse[cluster] = rmse
    all_results = pd.concat([all_results, result_df])

print("RMSE from each cluster:", clusters_rmse)

RMSE from each cluster: {2: 0.8709090833833981, 1: 0.9591603171535215, 3: 0.9515668812006262, 4: 0.8836018307304754, 5: 0.42056581286741207, 6: 0.5313983468603477, 7: 0.38217155591620017, 8: 0.8422656547470009}


In [101]:
overall_rmse = np.sqrt(mean_squared_error(all_results['stars'], all_results['predicted_stars']))
print("Overall RMSE across all clusters:", overall_rmse)

Overall RMSE across all clusters: 0.8478491926564184


In [ ]:
3: 0.9291631952891815, 4: 0.9611976256185978}

In [11]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

def optimize_cluster2(train_df, val_df):
    # 定义搜索空间
    space = {
        'max_depth': hp.choice('max_depth', [2, 3, 4]),
        'n_estimators': hp.choice('n_estimators', range(270, 310, 10)),
        'learning_rate': hp.uniform('learning_rate', 0.1, 0.15),
        'subsample': hp.uniform('subsample', 0.85, 0.92),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.8),  # 增加colsample_bytree进行搜索
        'gamma': hp.uniform('gamma', 0, 1),  # 增加gamma进行搜索
        'min_child_weight': hp.choice('min_child_weight', range(1, 6))  # 增加min_child_weight进行搜索
    }

    # 目标函数
    def objective(params):
        model = XGBRegressor(
            objective='reg:squarederror',
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            n_estimators=params['n_estimators'],
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            gamma=params['gamma'],
            min_child_weight=params['min_child_weight']
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        return {'loss': rmse, 'status': STATUS_OK}

    # 准备数据
    train_cluster_data = train_df[train_df['Cluster'] == 2]
    X_train = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_train = train_cluster_data['stars']

    val_cluster_data = val_df[val_df['Cluster'] == 2]
    X_val = val_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_val = val_cluster_data['stars']

    # 使用hyperopt进行优化
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=50,  # 这里设置迭代次数
        trials=trials
    )

    # 实例化和训练最佳模型
    best_model = XGBRegressor(
        objective='reg:squarederror',
        max_depth=int(best['max_depth']),
        learning_rate=best['learning_rate'],
        n_estimators=int(best['n_estimators']),
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        gamma=best['gamma'],
        min_child_weight=int(best['min_child_weight'])
    )
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_val)
    final_rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    # 输出结果
    print(f"Cluster 2 - 最佳参数: {best}")
    print(f"Cluster 2 - 最终测试集上的RMSE: {final_rmse}")
    return best_model, final_rmse

# 调用函数时，传入训练数据和验证数据
best_model, final_rmse = optimize_cluster2(train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.54trial/s, best loss: 0.8698640657701803]
Cluster 2 - 最佳参数: {'colsample_bytree': 0.682741441716062, 'gamma': 0.07673842656496505, 'learning_rate': 0.1232008650603654, 'max_depth': 0, 'min_child_weight': 4, 'n_estimators': 0, 'subsample': 0.8765024979544617}
Cluster 2 - 最终测试集上的RMSE: 3.4439417837227126


In [ ]:
    0: {'learning_rate': 0.04246778091879101, 'max_depth': 6, 'n_estimators': 491, 'subsample': 0.8689},
    2: {'learning_rate': 0.12350751460907078, 'max_depth': 3, 'n_estimators': 288, 'subsample': 0.8880},